In [28]:
import numpy as np
import pandas as pd
pd.set_option("display.precision", 2)


from pathlib import Path

git_repo = Path("CD11Research/")


Standard starting up Numpy and Pandas, Pandas is a data analysis tool built on top of Numpy. Display prediction is used to limit the amount of decimals Pandas shows.

Furthermore the folder which gets the data from the git repository is noted down. Below the data is read into Pandas DataFrames.

In [56]:
df_01IB = pd.read_csv(git_repo + '[01IB]_raw_inbound.txt')
df_01OB = pd.read_csv(git_repo + '[01OB]_raw_outbound.txt')

Lets look at the top of the data.

In [57]:
df_01IB.head()

,FLIGHT_ID,FLIGHT_ID_1,TRACK_ID,TRACK_ID_1,X,Y,MODE_C,CALLSIGN,ICAO_ACTYPE,DEST,ADEP,FLIGHT_TYPE,RADAR,LANDING_TIME,TIME,Unnamed: 15
0,5170097,5170097,135322667,135322667,9586,18498,400,KLM7451,B737,EHAM,ENGM,INBOUND,ARTACC,22-10-2010 0:51:33,22-10-2010 0:20:01,NaN
1,5170097,5170097,135322667,135322667,9562,18432,400,KLM7451,B737,EHAM,ENGM,INBOUND,ARTACC,22-10-2010 0:51:33,22-10-2010 0:20:06,NaN
2,5170097,5170097,135322667,135322667,9536,18362,400,KLM7451,B737,EHAM,ENGM,INBOUND,ARTACC,22-10-2010 0:51:33,22-10-2010 0:20:10,NaN
3,5170097,5170097,135322667,135322667,9512,18286,400,KLM7451,B737,EHAM,ENGM,INBOUND,ARTACC,22-10-2010 0:51:33,22-10-2010 0:20:15,NaN
4,5170097,5170097,135322667,135322667,9486,18216,400,KLM7451,B737,EHAM,ENGM,INBOUND,ARTACC,22-10-2010 0:51:33,22-10-2010 0:20:20,NaN


In [58]:
df_01OB.head()

,FLIGHT_ID,FLIGHT_ID_1,TRACK_ID,TRACK_ID_1,X,Y,MODE_C,CALLSIGN,ICAO_ACTYPE,DEST,ADEP,FLIGHT_TYPE,RADAR,TAKEOFF_TIME,TIME,Unnamed: 15
0,5170107,5170107,135323318,135323318,-8,-66,-1.0,SQC7377,B744,OMSJ,EHAM,OUTBOUND,ARTACC,22-10-2010 1:12:48,22-10-2010 1:12:38,NaN
1,5170107,5170107,135323318,135323318,-24,-78,-1.0,SQC7377,B744,OMSJ,EHAM,OUTBOUND,ARTACC,22-10-2010 1:12:48,22-10-2010 1:12:43,NaN
2,5170107,5170107,135323318,135323318,-44,-90,-1.0,SQC7377,B744,OMSJ,EHAM,OUTBOUND,ARTACC,22-10-2010 1:12:48,22-10-2010 1:12:48,NaN
3,5170107,5170107,135323318,135323318,-66,-102,-1.0,SQC7377,B744,OMSJ,EHAM,OUTBOUND,ARTACC,22-10-2010 1:12:48,22-10-2010 1:12:52,NaN
4,5170107,5170107,135323318,135323318,-88,-118,-1.0,SQC7377,B744,OMSJ,EHAM,OUTBOUND,ARTACC,22-10-2010 1:12:48,22-10-2010 1:12:57,NaN


We can see duplicate columns and an empty final column, lets fix that with usecols and all the columns we do want to use.

In [59]:
df_01IB = pd.read_csv(git_repo + '[01IB]_raw_inbound.txt', usecols=[0,2,4,5,6,7,8,9,10,11,12,13,14])
df_01OB = pd.read_csv(git_repo + '[01OB]_raw_outbound.txt', usecols=[0,2,4,5,6,7,8,9,10,11,12,13,14])

In [60]:
df_01OB.head()

,FLIGHT_ID,TRACK_ID,X,Y,MODE_C,CALLSIGN,ICAO_ACTYPE,DEST,ADEP,FLIGHT_TYPE,RADAR,TAKEOFF_TIME,TIME
0,5170107,135323318,-8,-66,-1.0,SQC7377,B744,OMSJ,EHAM,OUTBOUND,ARTACC,22-10-2010 1:12:48,22-10-2010 1:12:38
1,5170107,135323318,-24,-78,-1.0,SQC7377,B744,OMSJ,EHAM,OUTBOUND,ARTACC,22-10-2010 1:12:48,22-10-2010 1:12:43
2,5170107,135323318,-44,-90,-1.0,SQC7377,B744,OMSJ,EHAM,OUTBOUND,ARTACC,22-10-2010 1:12:48,22-10-2010 1:12:48
3,5170107,135323318,-66,-102,-1.0,SQC7377,B744,OMSJ,EHAM,OUTBOUND,ARTACC,22-10-2010 1:12:48,22-10-2010 1:12:52
4,5170107,135323318,-88,-118,-1.0,SQC7377,B744,OMSJ,EHAM,OUTBOUND,ARTACC,22-10-2010 1:12:48,22-10-2010 1:12:57


That is much better. Now we go on to split the data between training and prediction sets and make combined sets with inbound and outbound.


In [61]:
msk_IB = np.random.rand(len(df_01IB)) < 0.75

df_11IB = df_01IB[msk_IB]        #IB = inbound
df_11IBP = df_01IB[~msk_IB]      #P = prediction


#Wrong below. First use pandas group by to get flights.  'split-apply-combine'

np.random.seed(3141592)   #Keep this the same so every time this code runs, the resulting split will be the same, though the split itself is randomly chosen.
msk_OB = np.random.rand(len(df_01OB)) < 0.75

#Wrong above

df_11OB = df_01OB[msk_OB]        #OB = outbound
df_11OBP = df_01OB[~msk_OB]


df_11 = pd.concat([df_11IB, df_11OB], axis=0, ignore_index=True)
df_11P = pd.concat([df_11IBP, df_11OBP], axis=0, ignore_index=True)



Lets see if the bottom of the datasets look good:

In [64]:
df_11.tail()

,FLIGHT_ID,TRACK_ID,X,Y,MODE_C,CALLSIGN,ICAO_ACTYPE,DEST,ADEP,FLIGHT_TYPE,RADAR,LANDING_TIME,TIME,TAKEOFF_TIME
257624,5172119,135377047,11290,8878,254.0,JAE7454,B744,ZSPD,EHAM,OUTBOUND,ARTACC,NaN,22-10-2010 21:37:08,22-10-2010 21:16:13
257625,5172119,135377047,11364,8922,254.0,JAE7454,B744,ZSPD,EHAM,OUTBOUND,ARTACC,NaN,22-10-2010 21:37:12,22-10-2010 21:16:13
257626,5172119,135377047,11510,9012,256.0,JAE7454,B744,ZSPD,EHAM,OUTBOUND,ARTACC,NaN,22-10-2010 21:37:22,22-10-2010 21:16:13
257627,5172119,135377047,11658,9102,258.0,JAE7454,B744,ZSPD,EHAM,OUTBOUND,ARTACC,NaN,22-10-2010 21:37:32,22-10-2010 21:16:13
257628,5172119,135377047,11732,9148,259.0,JAE7454,B744,ZSPD,EHAM,OUTBOUND,ARTACC,NaN,22-10-2010 21:37:36,22-10-2010 21:16:13


In [65]:
df_11P.tail()

,FLIGHT_ID,TRACK_ID,X,Y,MODE_C,CALLSIGN,ICAO_ACTYPE,DEST,ADEP,FLIGHT_TYPE,RADAR,LANDING_TIME,TIME,TAKEOFF_TIME
86153,5172119,135377047,11438,8966,255.0,JAE7454,B744,ZSPD,EHAM,OUTBOUND,ARTACC,NaN,22-10-2010 21:37:17,22-10-2010 21:16:13
86154,5172119,135377047,11584,9056,257.0,JAE7454,B744,ZSPD,EHAM,OUTBOUND,ARTACC,NaN,22-10-2010 21:37:27,22-10-2010 21:16:13
86155,5172119,135377047,11804,9194,260.0,JAE7454,B744,ZSPD,EHAM,OUTBOUND,ARTACC,NaN,22-10-2010 21:37:41,22-10-2010 21:16:13
86156,5172119,135377047,11878,9238,261.0,JAE7454,B744,ZSPD,EHAM,OUTBOUND,ARTACC,NaN,22-10-2010 21:37:46,22-10-2010 21:16:13
86157,5172119,135377047,11952,9284,261.0,JAE7454,B744,ZSPD,EHAM,OUTBOUND,ARTACC,NaN,22-10-2010 21:37:51,22-10-2010 21:16:13


Now outputting them to files.

In [66]:
df_11.to_pickle(git_repo + '[11].pkl')
df_11P.to_pickle(git_repo + '[11P].pkl')

First level data sets have now been made and put out as pickles.